In [2]:
import glob
import os
import numpy as np
import pandas as pd
import zipfile
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

In [41]:
#reads january 2010 (for playing around)
path = "C:\\Users\\linathan\\Documents\\bts_downloads_2010_2023"
all_files = [path+"\\On_Time_Reporting_Carrier_On_Time_Performance_1987_present_2010_1"]
name = "On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2010_1"

In [42]:
#reads january 2010 (for playing around)
print(all_files[0])
zf = zipfile.ZipFile(all_files[0]+'.zip')
print(zf.namelist())
df = pd.read_csv(zf.open(name+'.csv'))

C:\Users\linathan\Documents\bts_downloads_2010_2023\On_Time_Reporting_Carrier_On_Time_Performance_1987_present_2015_5
['On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2015_5.csv', 'readme.html']


C:\Users\linathan\AppData\Local\Temp\1\ipykernel_17988\918730203.py:4: DtypeWarning: Columns (77,84) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(zf.open(name+'.csv'))


In [52]:
#creates a dictionary where every (year, month) corresponds to the csv file name for that year and month
fileBook = {}
for year in range(2010, 2024):
    maxMonth = 12 if year == 2023 else 13 
    for month in range(1, maxMonth):
        csvName = f"On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_{year}_{month}"
        fileBook[(year, month)] = csvName

In [53]:
#creates a dictionary where every (year, month) corresponds to the DataFrame for that year and month
#currently runs into memory issues even on Joao's office desktop
dataBook = {}
for year in range(2010,2024):
    maxMonth = 12 if year == 2023 else 13
    for month in range(1, maxMonth):
        zf = zipfile.ZipFile(path + f'\\On_Time_Reporting_Carrier_On_Time_Performance_1987_present_{year}_{month}' + '.zip')
        dataBook[(year, month)] = pd.read_csv(zf.open(fileBook[(year, month)] + '.csv'))

C:\Users\linathan\AppData\Local\Temp\1\ipykernel_17988\2943428216.py:6: DtypeWarning: Columns (77,84) have mixed types. Specify dtype option on import or set low_memory=False.
  dataBook[(year, month)] = pd.read_csv(zf.open(fileBook[(year, month)] + '.csv'))
C:\Users\linathan\AppData\Local\Temp\1\ipykernel_17988\2943428216.py:6: DtypeWarning: Columns (77,84) have mixed types. Specify dtype option on import or set low_memory=False.
  dataBook[(year, month)] = pd.read_csv(zf.open(fileBook[(year, month)] + '.csv'))
C:\Users\linathan\AppData\Local\Temp\1\ipykernel_17988\2943428216.py:6: DtypeWarning: Columns (69,76,77,84) have mixed types. Specify dtype option on import or set low_memory=False.
  dataBook[(year, month)] = pd.read_csv(zf.open(fileBook[(year, month)] + '.csv'))
C:\Users\linathan\AppData\Local\Temp\1\ipykernel_17988\2943428216.py:6: DtypeWarning: Columns (77,84) have mixed types. Specify dtype option on import or set low_memory=False.
  dataBook[(year, month)] = pd.read_csv(z

MemoryError: Unable to allocate 282. MiB for an array with shape (70, 528588) and data type float64

In [24]:
#removes rows with diverted flights
df = df[df['Diverted']== 0]

In [26]:
#creates new row which is the sum of non-weather delay times
df['NotWeatherDelay'] = df['CarrierDelay'] + df['NASDelay'] + df['LateAircraftDelay'] + df['SecurityDelay']

C:\Users\linathan\AppData\Local\Temp\1\ipykernel_17988\565079504.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['NotWeatherDelay'] = df['CarrierDelay'] + df['NASDelay'] + df['LateAircraftDelay'] + df['SecurityDelay']


In [43]:
df.Reporting_Airline.unique()

array(['UA', 'US', 'VX', 'WN', 'NK', 'OO', 'EV', 'HA', 'MQ', 'DL', 'B6',
       'F9', 'AA', 'AS'], dtype=object)

In [27]:
#keeps passthrough columns, drops any columns not mentioned
#one hot encoder for Reporting_Airline, Origin, Dest
ct = ColumnTransformer([
    ('pass', 'passthrough', ['Year', 'Month', 'DayofMonth',
    'DayofWeek','CRSDepTime','DepTime',
    'DepDelay','DepDelayMax','DepDel15','CRSArrTime','ArrTime','ArrDelay','ArrDelayMax','ArrDel15',
    'Cancelled','AirTime', 'WeatherDelay', 'NotWeatherDelay']),
    ('1hotID', OneHotEncoder(), ['Dest', 'Origin', 'Reporting_Airline'])
])